## Model

In [17]:
from torch import nn
from torch.nn import functional as F
import torch
import torch.nn.init as init
from torchvision import models
import time

class OURS(nn.Module):
    def __init__(self, num_class=7,num_head=4, pretrained=True):
        super(OURS, self).__init__()
        resnet = models.resnet18(pretrained)
        
        if pretrained:
            # resnet 18 og model
            checkpoint = torch.load('./latent-ofer/models/resnet18_msceleb.pth')
            resnet.load_state_dict(checkpoint['state_dict'],strict=True)

            

        self.features = nn.Sequential(*list(resnet.children())[:-2])
        self.num_head = num_head
        for i in range(num_head):
            setattr(self,"cat_head%d" %i, CrossAttentionHead())
        self.sig = nn.Sigmoid()
        self.hh_layer = nn.Linear(39936, 10000)
        self.hh_layer2 = nn.Linear(10000, 5000)
        self.hh_batch1 = nn.BatchNorm1d(5000)
        self.hh_layer3 = nn.Linear(5000, 1024)
        self.hh_layer4 = nn.Linear(1024, 256)
        self.batch_norm1 = nn.BatchNorm1d(256)


        self.fc = nn.Linear(512+256, 256)
        self.fc2 = nn.Linear(256, 128)
        self.batch_norm = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, num_class)
        self.bn = nn.BatchNorm1d(num_class)


    def forward(self, x, latent):

        x = self.features(x)
        heads = []
        for i in range(self.num_head):
            heads.append(getattr(self,"cat_head%d" %i)(x))
        
        heads = torch.stack(heads).permute([1,0,2])
        if heads.size(1)>1:
            heads = F.log_softmax(heads,dim=1)
            heads = heads.sum(dim=1)

        latent = self.hh_layer(latent)
        latent = self.hh_layer2(latent)
        latent = self.hh_batch1(latent)
        latent = self.hh_layer3(latent)
        latent = self.hh_layer4(latent)
        latent = self.batch_norm1(latent)
        
        out = torch.cat([heads, latent], dim=1)
        out = self.fc(out)
        out = self.fc2(out)
        out = self.batch_norm(out)
        out = self.fc4(out)
        out = self.bn(out)
   
        return out, x, heads

class CrossAttentionHead(nn.Module):
    def __init__(self):
        super().__init__()
        self.sa = SpatialAttention()
        self.ca = ChannelAttention()
        self.init_weights()


    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)         
    def forward(self, x):
        ca = self.ca(x)
        sa = self.sa(ca)

        return sa


class SpatialAttention(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1x1 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1),
            nn.BatchNorm2d(256),
        )
        self.conv_3x3 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
        )
        self.conv_1x3 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=(1,3),padding=(0,1)),
            nn.BatchNorm2d(512),
        )
        self.conv_3x1 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=(3,1),padding=(1,0)),
            nn.BatchNorm2d(512),
        )
        self.relu = nn.ReLU()
        self.gap = nn.AdaptiveAvgPool2d(1)


    def forward(self, x):
        y = self.conv1x1(x)
        a = self.conv_3x3(y)
        b = self.conv_1x3(y)
        c = self.conv_3x1(y)
        
        y = self.relu(a + b + c)
        y = y.sum(dim=1,keepdim=True) 

        out = x*y
        out = self.gap(out)
        out = out.view(out.size(0), -1)
        return out 

class ChannelAttention(nn.Module):

    def __init__(self):
        super().__init__()
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.attention = nn.Sequential(
            nn.Linear(512, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 512),
            nn.Sigmoid()    
        )


    def forward(self, sa):
        sa2 = self.gap(sa)
        sa2 = sa2.view(sa2.size(0),-1)
        y = self.attention(sa2)
        y = y.unsqueeze(dim = -1)
        y = y.unsqueeze(dim = -1)

        out = sa * y
        return out

## Latent-OFER

In [2]:
import os
import warnings
from tqdm import tqdm

from PIL import Image
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision import transforms

from sklearn.metrics import balanced_accuracy_score

def warn(*args, **kwargs):
    pass
warnings.warn = warn

class FERPlusDataset(data.Dataset):
    def __init__(self, data_csv, phase, transform=None):
        self.phase = phase
        self.transform = transform

        # Read the dataset CSV file
        self.data = pd.read_csv(data_csv)
        self.data.iloc[:, 2:12] = self.data.iloc[:, 2:12].replace(1, 0)
        # Get file paths and labels
        self.file_paths = self.data.iloc[:, 0].values
        self.counts = self.data.iloc[:, 2:12].values  # 감정 점수들

        # Apply constraints to filter valid samples
        self._apply_constraints()

        # Use argmax to determine the emotion class
        self.labels = np.argmax(self.counts, axis=1)

        # Debugging: Check label range
        print("Unique labels in dataset after filtering:", np.unique(self.labels))

    def _apply_constraints(self):
        # Constraint : 'unknown-face' 또는 'not-face' 레이블 제거
        max_counts = self.counts.max(axis=1)
        counts_eq_max = (self.counts == max_counts[:, None])
        constraint1_violation = counts_eq_max[:, [8, 9]].any(axis=1)

        # Constraint : 1인 라벨 0으로 만들기


        # Constraint : 최대 투표 수를 가진 레이블이 3개 초과 제거
        num_max_labels = counts_eq_max.sum(axis=1)
        constraint2_violation = num_max_labels > 3

        # Constraint : 최대 투표 수가 전체 투표 수의 절반 이하인 경우 제거
        total_votes = self.counts.sum(axis=1)
        constraint3_violation = max_counts <= (total_votes / 2)

        # Combine constraints
        valid_samples = ~(
            constraint1_violation | constraint2_violation | constraint3_violation
        )

        # Apply valid samples filter
        self.file_paths = self.file_paths[valid_samples]
        self.counts = self.counts[valid_samples]

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        if self.phase == 'train':
            path = '/data/FER2013/FER2013Train/' + self.file_paths[idx]
        elif self.phase == 'val':
            path = '/data/FER2013/FER2013Valid/' + self.file_paths[idx]
        elif self.phase == 'test':
            path = '/data/FER2013/FER2013Test/' + self.file_paths[idx]
        image = Image.open(path).convert('RGB')
        label = self.labels[idx]

        if self.transform is not None:
            image = self.transform(image)

        return image, label



# Define variables
train_csv = '/data/FER2013/train_label.csv'
val_csv = '/data/FER2013/valid_label.csv'
test_csv = '/data/FER2013/test_label.csv'
batch_size = 128
lr = 0.1
workers = 4
epochs = 60

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = True

model = OURS(num_class=8 ,num_head=4, pretrained=True)  # YOUR MODEL CLASS SHOULD BE DEFINED ELSEWHERE
model.to(device)

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([
            transforms.RandomRotation(20),
            transforms.RandomCrop(224, padding=32)
        ], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(scale=(0.02, 0.25)),
])

train_dataset = FERPlusDataset(train_csv, phase='train', transform=data_transforms)
print('Whole train set size:', train_dataset.__len__())

train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=batch_size,
                                            num_workers=workers,
                                            shuffle=True,
                                            pin_memory=True)

data_transforms_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])

val_dataset = FERPlusDataset(val_csv, phase='val', transform=data_transforms_val)
print('Validation set size:', val_dataset.__len__())

val_loader = torch.utils.data.DataLoader(val_dataset,
                                            batch_size=batch_size,
                                            num_workers=workers,
                                            shuffle=False,
                                            pin_memory=True)

criterion_cls = torch.nn.CrossEntropyLoss()

params = list(model.parameters())
optimizer = torch.optim.SGD(params, lr=lr, weight_decay=1e-4, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

best_acc = 0
for epoch in tqdm(range(1, epochs + 1)):
    running_loss = 0.0
    correct_sum = 0
    iter_cnt = 0
    model.train()

    for imgs, targets in train_loader:
        iter_cnt += 1
        optimizer.zero_grad()

        imgs = imgs.to(device)
        targets = targets.to(device)
        out, _, _ = model(imgs, torch.zeros(imgs.size(0), 39936).to(device)) # Pass None for latent

        loss = criterion_cls(out, targets)

        loss.backward()
        optimizer.step()

        running_loss += loss
        _, predicts = torch.max(out, 1)
        correct_num = torch.eq(predicts, targets).sum()
        correct_sum += correct_num

    acc = correct_sum.float() / float(train_dataset.__len__())
    running_loss = running_loss / iter_cnt
    tqdm.write('[Epoch %d] Training accuracy: %.4f. Loss: %.3f. LR %.6f' % (epoch, acc, running_loss, optimizer.param_groups[0]['lr']))

    with torch.no_grad():
        running_loss = 0.0
        iter_cnt = 0
        bingo_cnt = 0
        sample_cnt = 0
        baccs = []

        y_true = []
        y_pred = []

        model.eval()
        for imgs, targets in val_loader:
            imgs = imgs.to(device)
            targets = targets.to(device)

            out, _, _ = model(imgs, torch.zeros(imgs.size(0), 39936).to(device))   # Pass None for latent
            loss = criterion_cls(out, targets)
            running_loss += loss
            iter_cnt += 1
            _, predicts = torch.max(out, 1)
            correct_num = torch.eq(predicts, targets)
            bingo_cnt += correct_num.sum().cpu()
            sample_cnt += out.size(0)
            y_true.append(targets.cpu().numpy())
            y_pred.append(predicts.cpu().numpy())

            baccs.append(balanced_accuracy_score(targets.cpu().numpy(), predicts.cpu().numpy()))
        running_loss = running_loss / iter_cnt
        scheduler.step()

        acc = bingo_cnt.float() / float(sample_cnt)
        acc = np.around(acc.numpy(), 4)
        best_acc = max(acc, best_acc)

        y_true = np.concatenate(y_true)
        y_pred = np.concatenate(y_pred)

        bacc = np.around(np.mean(baccs), 4)
        tqdm.write("[Epoch %d] Validation accuracy:%.4f. bacc:%.4f. Loss:%.3f" % (epoch, acc, bacc, running_loss))
        tqdm.write("best_acc:" + str(best_acc))

        #if acc > 0.86 and acc == best_acc:
        #    torch.save({'iter': epoch,
        #                'model_state_dict': model.state_dict(),
        #                 'optimizer_state_dict': optimizer.state_dict()},
        #                os.path.join('checkpoints', "ferplus_epoch" + str(epoch) + "_acc" + str(acc) + "_bacc" + str(bacc) + ".pth"))
        #    tqdm.write('Model saved.')

Unique labels in dataset after filtering: [0 1 2 3 4 5 6 7]
Whole train set size: 25045
Unique labels in dataset after filtering: [0 1 2 3 4 5 6 7]
Validation set size: 3191


  0%|          | 0/60 [00:22<?, ?it/s]

[Epoch 1] Training accuracy: 0.7271. Loss: 0.781. LR 0.100000


  2%|▏         | 1/60 [00:24<24:13, 24.63s/it]

[Epoch 1] Validation accuracy:0.7922. bacc:0.6051. Loss:0.588
best_acc:0.7922


...

[Epoch 59] Validation accuracy:0.8793. bacc:0.7575. Loss:0.379
best_acc:0.8825


 98%|█████████▊| 59/60 [22:14<00:22, 22.21s/it]

[Epoch 60] Training accuracy: 0.9653. Loss: 0.104. LR 0.000001


100%|██████████| 60/60 [22:16<00:00, 22.28s/it]

[Epoch 60] Validation accuracy:0.8762. bacc:0.7587. Loss:0.386
best_acc:0.8825


In [15]:
def evaluate_model(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    y_true = []
    y_pred = []

    print("Starting evaluation...")
    with torch.no_grad():  # Disable gradient computation
        for i, (imgs, targets) in enumerate(test_loader):

            imgs = imgs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs, _, _ = model(imgs, torch.zeros(imgs.size(0), 39936).to(device))  # Provide dummy latent input
            _, predictions = torch.max(outputs, 1)  # Get predicted class

            # Collect results
            correct += (predictions == targets).sum().item()
            total += targets.size(0)

            y_true.extend(targets.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())

    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0  # Prevent division by zero
    print(f"Test Accuracy: {accuracy}")  
    print(f"Test Accuracy: {accuracy}")
    print(f"Test Accuracy: {accuracy}")

    return accuracy
data_transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])

test_dataset = FERPlusDataset(test_csv, phase='test', transform=data_transforms_test)
print('Test set size:', test_dataset.__len__())

test_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size=batch_size,
                                            num_workers=workers,
                                            shuffle=False,
                                            pin_memory=True)

# Evaluate model
acc = evaluate_model(model, test_loader, device)
print(f"Final Test Accuracy: {acc:.4f}")


Unique labels in dataset after filtering: [0 1 2 3 4 5 6 7]
Test set size: 3136
Starting evaluation...


Test Accuracy: 0.8788265306122449
Test Accuracy: 0.8788265306122449
Test Accuracy: 0.8788265306122449
Final Test Accuracy: 0.8788


## Latent-OFER + dlib

In [24]:
import dlib
import pandas as pd
import numpy as np
from PIL import Image
import cv2
from torch.utils.data import Dataset

class FERPlusDataset(Dataset):
    def __init__(self, data_csv, phase, transform=None):
        self.phase = phase
        self.transform = transform

        # Read the dataset CSV file
        self.data = pd.read_csv(data_csv)
        self.data.iloc[:, 2:12] = self.data.iloc[:, 2:12].replace(1, 0)

        # Get file paths and labels
        self.file_paths = self.data.iloc[:, 0].values
        self.counts = self.data.iloc[:, 2:12].values  # 감정 점수들

        # Apply constraints to filter valid samples
        self._apply_constraints()

        # Use argmax to determine the emotion class
        self.labels = np.argmax(self.counts, axis=1)

        # Debugging: Check label range
        print("Unique labels in dataset after filtering:", np.unique(self.labels))

        # Dlib face detector and predictor initialization
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor('/root/FER2013/shape_predictor_68_face_landmarks.dat')  # Dlib 모델 파일 경로 필요

    def _apply_constraints(self):
        # Constraint : 'unknown-face' 또는 'not-face' 레이블 제거
        max_counts = self.counts.max(axis=1)
        counts_eq_max = (self.counts == max_counts[:, None])
        constraint1_violation = counts_eq_max[:, [8, 9]].any(axis=1)

        # Constraint : 최대 투표 수를 가진 레이블이 3개 초과 제거
        num_max_labels = counts_eq_max.sum(axis=1)
        constraint2_violation = num_max_labels > 3

        # Constraint : 최대 투표 수가 전체 투표 수의 절반 이하인 경우 제거
        total_votes = self.counts.sum(axis=1)
        constraint3_violation = max_counts <= (total_votes / 2)

        # Combine constraints
        valid_samples = ~(
            constraint1_violation | constraint2_violation | constraint3_violation
        )

        # Apply valid samples filter
        self.file_paths = self.file_paths[valid_samples]
        self.counts = self.counts[valid_samples]

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        if self.phase == 'train':
            path = '/data/FER2013/FER2013Train/' + self.file_paths[idx]
        elif self.phase == 'val':
            path = '/data/FER2013/FER2013Valid/' + self.file_paths[idx]
        elif self.phase == 'test':
            path = '/data/FER2013/FER2013Test/' + self.file_paths[idx]
        
        # Open image
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Align face using Dlib
        image = self._align_face(image)

        # Convert to PIL image for further processing
        image = Image.fromarray(image)

        label = self.labels[idx]

        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def _align_face(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        faces = self.detector(gray)

        if len(faces) == 0:
            return image  # 얼굴이 감지되지 않으면 원본 이미지를 반환

        for face in faces:
            landmarks = self.predictor(gray, face)

            # 좌우 눈의 중심 좌표 추출
            left_eye = (landmarks.part(36).x, landmarks.part(36).y)
            right_eye = (landmarks.part(45).x, landmarks.part(45).y)

            # 두 눈의 중심 계산
            eye_center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)

            # 눈 사이의 기울기 계산
            delta_x = right_eye[0] - left_eye[0]
            delta_y = right_eye[1] - left_eye[1]
            angle = np.degrees(np.arctan2(delta_y, delta_x))

            # 회전 행렬 계산
            rot_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)

            # 이미지 회전 및 정렬
            aligned_face = cv2.warpAffine(image, rot_matrix, (image.shape[1], image.shape[0]))

            return aligned_face  # 첫 번째 얼굴만 처리

        return image


In [25]:


# Define variables
train_csv = '/data/FER2013/train_label.csv'
val_csv = '/data/FER2013/valid_label.csv'
test_csv = '/data/FER2013/test_label.csv'
batch_size = 128
lr = 0.1
workers = 4
epochs = 60

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = True

model = OURS(num_class=8 ,num_head=4, pretrained=True)  # YOUR MODEL CLASS SHOULD BE DEFINED ELSEWHERE
model.to(device)

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([
            transforms.RandomRotation(20),
            transforms.RandomCrop(224, padding=32)
        ], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(scale=(0.02, 0.25)),
])

train_dataset = FERPlusDataset(train_csv, phase='train', transform=data_transforms)
print('Whole train set size:', train_dataset.__len__())

train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=batch_size,
                                            num_workers=workers,
                                            shuffle=True,
                                            pin_memory=True)

data_transforms_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])

val_dataset = FERPlusDataset(val_csv, phase='val', transform=data_transforms_val)
print('Validation set size:', val_dataset.__len__())

val_loader = torch.utils.data.DataLoader(val_dataset,
                                            batch_size=batch_size,
                                            num_workers=workers,
                                            shuffle=False,
                                            pin_memory=True)

criterion_cls = torch.nn.CrossEntropyLoss()

params = list(model.parameters())
optimizer = torch.optim.SGD(params, lr=lr, weight_decay=1e-4, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

best_acc = 0
for epoch in tqdm(range(1, epochs + 1)):
    running_loss = 0.0
    correct_sum = 0
    iter_cnt = 0
    model.train()

    for imgs, targets in train_loader:
        iter_cnt += 1
        optimizer.zero_grad()

        imgs = imgs.to(device)
        targets = targets.to(device)
        out, _, _ = model(imgs, torch.zeros(imgs.size(0), 39936).to(device)) # Pass None for latent

        loss = criterion_cls(out, targets)

        loss.backward()
        optimizer.step()

        running_loss += loss
        _, predicts = torch.max(out, 1)
        correct_num = torch.eq(predicts, targets).sum()
        correct_sum += correct_num

    acc = correct_sum.float() / float(train_dataset.__len__())
    running_loss = running_loss / iter_cnt
    tqdm.write('[Epoch %d] Training accuracy: %.4f. Loss: %.3f. LR %.6f' % (epoch, acc, running_loss, optimizer.param_groups[0]['lr']))

    with torch.no_grad():
        running_loss = 0.0
        iter_cnt = 0
        bingo_cnt = 0
        sample_cnt = 0
        baccs = []

        y_true = []
        y_pred = []

        model.eval()
        for imgs, targets in val_loader:
            imgs = imgs.to(device)
            targets = targets.to(device)

            out, _, _ = model(imgs, torch.zeros(imgs.size(0), 39936).to(device))   # Pass None for latent
            loss = criterion_cls(out, targets)
            running_loss += loss
            iter_cnt += 1
            _, predicts = torch.max(out, 1)
            correct_num = torch.eq(predicts, targets)
            bingo_cnt += correct_num.sum().cpu()
            sample_cnt += out.size(0)
            y_true.append(targets.cpu().numpy())
            y_pred.append(predicts.cpu().numpy())

            baccs.append(balanced_accuracy_score(targets.cpu().numpy(), predicts.cpu().numpy()))
        running_loss = running_loss / iter_cnt
        scheduler.step()

        acc = bingo_cnt.float() / float(sample_cnt)
        acc = np.around(acc.numpy(), 4)
        best_acc = max(acc, best_acc)

        y_true = np.concatenate(y_true)
        y_pred = np.concatenate(y_pred)

        bacc = np.around(np.mean(baccs), 4)
        tqdm.write("[Epoch %d] Validation accuracy:%.4f. bacc:%.4f. Loss:%.3f" % (epoch, acc, bacc, running_loss))
        tqdm.write("best_acc:" + str(best_acc))

Unique labels in dataset after filtering: [0 1 2 3 4 5 6 7]
Whole train set size: 25045
Unique labels in dataset after filtering: [0 1 2 3 4 5 6 7]
Validation set size: 3191


  0%|          | 0/60 [00:20<?, ?it/s]

[Epoch 1] Training accuracy: 0.7320. Loss: 0.769. LR 0.100000


  2%|▏         | 1/60 [00:22<22:15, 22.63s/it]

[Epoch 1] Validation accuracy:0.8270. bacc:0.6169. Loss:0.494
best_acc:0.827


...

[Epoch 59] Validation accuracy:0.8819. bacc:0.7769. Loss:0.382
best_acc:0.8847


 98%|█████████▊| 59/60 [22:16<00:22, 22.29s/it]

[Epoch 60] Training accuracy: 0.9630. Loss: 0.107. LR 0.000001


100%|██████████| 60/60 [22:18<00:00, 22.30s/it]

[Epoch 60] Validation accuracy:0.8812. bacc:0.7754. Loss:0.386
best_acc:0.8847


In [26]:
def evaluate_model(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    y_true = []
    y_pred = []

    print("Starting evaluation...")
    with torch.no_grad():  # Disable gradient computation
        for i, (imgs, targets) in enumerate(test_loader):

            imgs = imgs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs, _, _ = model(imgs, torch.zeros(imgs.size(0), 39936).to(device))  # Provide dummy latent input
            _, predictions = torch.max(outputs, 1)  # Get predicted class

            # Collect results
            correct += (predictions == targets).sum().item()
            total += targets.size(0)

            y_true.extend(targets.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())

    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0  # Prevent division by zero
    print(f"Test Accuracy: {accuracy}")  
    print(f"Test Accuracy: {accuracy}")
    print(f"Test Accuracy: {accuracy}")

    return accuracy
data_transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])

test_dataset = FERPlusDataset(test_csv, phase='test', transform=data_transforms_test)
print('Test set size:', test_dataset.__len__())

test_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size=batch_size,
                                            num_workers=workers,
                                            shuffle=False,
                                            pin_memory=True)

# Evaluate model
acc = evaluate_model(model, test_loader, device)
print(f"Final Test Accuracy: {acc:.4f}")


Unique labels in dataset after filtering: [0 1 2 3 4 5 6 7]
Test set size: 3136
Starting evaluation...
Test Accuracy: 0.8807397959183674
Test Accuracy: 0.8807397959183674
Test Accuracy: 0.8807397959183674
Final Test Accuracy: 0.8807


# AffectNet Latent-OFER Dlib

In [1]:
from torch import nn
from torch.nn import functional as F
import torch
import torch.nn.init as init
from torchvision import models
import time
from torchvision import transforms
from sklearn.metrics import balanced_accuracy_score

class OURS(nn.Module):
    def __init__(self, num_class=7, num_head=4, affectnet_weights_path=None):
        super(OURS, self).__init__()
        resnet = models.resnet50(pretrained=False)

        if affectnet_weights_path:
            checkpoint = torch.load(affectnet_weights_path, map_location=torch.device('cpu'))
            if 'state_dict' in checkpoint:
                resnet.load_state_dict(checkpoint['state_dict'], strict=False)
            elif 'model_state_dict' in checkpoint:
                resnet.load_state_dict(checkpoint['model_state_dict'], strict=False)
            else:
                resnet.load_state_dict(checkpoint, strict=False)
            print("AffectNet weights loaded successfully.")

        self.features = nn.Sequential(*list(resnet.children())[:-2])
        self.num_head = num_head
        for i in range(num_head):
            setattr(self, f"cat_head{i}", CrossAttentionHead())

        self.sig = nn.Sigmoid()
        self.hh_layer = nn.Linear(39936, 1024)  # latent 크기 조정
        self.fc = nn.Linear(2048 + 1024, 256)  # heads + latent 결합 크기 반영
        self.fc2 = nn.Linear(256, 128)
        self.batch_norm = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, num_class)
        self.bn = nn.BatchNorm1d(num_class)

    def forward(self, x, latent):
        x = self.features(x)
        heads = []
        for i in range(self.num_head):
            heads.append(getattr(self, f"cat_head{i}")(x))

        # heads 결합
        heads = torch.stack(heads).permute([1, 0, 2])  # (batch_size, num_heads, features)
        heads = heads.sum(dim=1)  # (batch_size, features)

        # latent 변환
        latent = self.hh_layer(latent)

        # heads와 latent 결합
        out = torch.cat([heads, latent], dim=1)  # (batch_size, 2048 + 1024)
        out = self.fc(out)
        out = self.fc2(out)
        out = self.batch_norm(out)
        out = self.fc4(out)
        out = self.bn(out)

        return out, x, heads


class CrossAttentionHead(nn.Module):
    def __init__(self):
        super().__init__()
        self.sa = SpatialAttention()
        self.ca = ChannelAttention()
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x):
        ca = self.ca(x)
        sa = self.sa(ca)
        return sa

class SpatialAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1x1 = nn.Sequential(
            nn.Conv2d(2048, 1024, kernel_size=1),  # 2048 -> 1024로 축소
            nn.BatchNorm2d(1024),
        )
        self.conv_3x3 = nn.Sequential(
            nn.Conv2d(1024, 2048, kernel_size=3, padding=1),  # 1024 -> 2048로 복구
            nn.BatchNorm2d(2048),
        )
        self.conv_1x3 = nn.Sequential(
            nn.Conv2d(1024, 2048, kernel_size=(1, 3), padding=(0, 1)),
            nn.BatchNorm2d(2048),
        )
        self.conv_3x1 = nn.Sequential(
            nn.Conv2d(1024, 2048, kernel_size=(3, 1), padding=(1, 0)),
            nn.BatchNorm2d(2048),
        )
        self.relu = nn.ReLU()
        self.gap = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        y = self.conv1x1(x)
        a = self.conv_3x3(y)
        b = self.conv_1x3(y)
        c = self.conv_3x1(y)

        y = self.relu(a + b + c)
        y = y.sum(dim=1, keepdim=True)

        out = x * y
        out = self.gap(out)
        out = out.view(out.size(0), -1)
        return out


class ChannelAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.attention = nn.Sequential(
            nn.Linear(2048, 512),  # 2048 -> 512로 축소
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 2048),  # 512 -> 2048로 복구
            nn.Sigmoid()
        )

    def forward(self, sa):
        sa2 = self.gap(sa)  # (batch_size, 2048, 1, 1)
        sa2 = sa2.view(sa2.size(0), -1)  # (batch_size, 2048)
        y = self.attention(sa2)  # (batch_size, 2048)
        y = y.unsqueeze(dim=-1).unsqueeze(dim=-1)  # (batch_size, 2048, 1, 1)

        out = sa * y
        return out



In [2]:
import dlib
import pandas as pd
import numpy as np
from PIL import Image
import cv2
from torch.utils.data import Dataset
from tqdm import tqdm
class FERPlusDataset(Dataset):
    def __init__(self, data_csv, phase, transform=None):
        self.phase = phase
        self.transform = transform

        # Read the dataset CSV file
        self.data = pd.read_csv(data_csv)
        self.data.iloc[:, 2:12] = self.data.iloc[:, 2:12].replace(1, 0)

        # Get file paths and labels
        self.file_paths = self.data.iloc[:, 0].values
        self.counts = self.data.iloc[:, 2:12].values  # 감정 점수들

        # Apply constraints to filter valid samples
        self._apply_constraints()

        # Use argmax to determine the emotion class
        self.labels = np.argmax(self.counts, axis=1)

        # Debugging: Check label range
        print("Unique labels in dataset after filtering:", np.unique(self.labels))

        # Dlib face detector and predictor initialization
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor('/root/FER2013/shape_predictor_68_face_landmarks.dat')  # Dlib 모델 파일 경로 필요

    def _apply_constraints(self):
        # Constraint : 'unknown-face' 또는 'not-face' 레이블 제거
        max_counts = self.counts.max(axis=1)
        counts_eq_max = (self.counts == max_counts[:, None])
        constraint1_violation = counts_eq_max[:, [8, 9]].any(axis=1)

        # Constraint : 최대 투표 수를 가진 레이블이 3개 초과 제거
        num_max_labels = counts_eq_max.sum(axis=1)
        constraint2_violation = num_max_labels > 3

        # Constraint : 최대 투표 수가 전체 투표 수의 절반 이하인 경우 제거
        total_votes = self.counts.sum(axis=1)
        constraint3_violation = max_counts <= (total_votes / 2)

        # Combine constraints
        valid_samples = ~(
            constraint1_violation | constraint2_violation | constraint3_violation
        )

        # Apply valid samples filter
        self.file_paths = self.file_paths[valid_samples]
        self.counts = self.counts[valid_samples]

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        if self.phase == 'train':
            path = '/data/FER2013/FER2013Train/' + self.file_paths[idx]
        elif self.phase == 'val':
            path = '/data/FER2013/FER2013Valid/' + self.file_paths[idx]
        elif self.phase == 'test':
            path = '/data/FER2013/FER2013Test/' + self.file_paths[idx]
        
        # Open image
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Align face using Dlib
        image = self._align_face(image)

        # Convert to PIL image for further processing
        image = Image.fromarray(image)

        label = self.labels[idx]

        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def _align_face(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        faces = self.detector(gray)

        if len(faces) == 0:
            return image  # 얼굴이 감지되지 않으면 원본 이미지를 반환

        for face in faces:
            landmarks = self.predictor(gray, face)

            # 좌우 눈의 중심 좌표 추출
            left_eye = (landmarks.part(36).x, landmarks.part(36).y)
            right_eye = (landmarks.part(45).x, landmarks.part(45).y)

            # 두 눈의 중심 계산
            eye_center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)

            # 눈 사이의 기울기 계산
            delta_x = right_eye[0] - left_eye[0]
            delta_y = right_eye[1] - left_eye[1]
            angle = np.degrees(np.arctan2(delta_y, delta_x))

            # 회전 행렬 계산
            rot_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)

            # 이미지 회전 및 정렬
            aligned_face = cv2.warpAffine(image, rot_matrix, (image.shape[1], image.shape[0]))

            return aligned_face  # 첫 번째 얼굴만 처리

        return image


In [3]:


# Define variables
train_csv = '/data/FER2013/train_label.csv'
val_csv = '/data/FER2013/valid_label.csv'
test_csv = '/data/FER2013/test_label.csv'
batch_size = 128
lr = 0.1
workers = 4
epochs = 60

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = True

model = OURS(num_class=8 ,num_head=4, affectnet_weights_path='/root/FER2013/FER_static_ResNet50_AffectNet.pt')# YOUR MODEL CLASS SHOULD BE DEFINED ELSEWHERE
model.to(device)

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([
            transforms.RandomRotation(20),
            transforms.RandomCrop(224, padding=32)
        ], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(scale=(0.02, 0.25)),
])

train_dataset = FERPlusDataset(train_csv, phase='train', transform=data_transforms)
print('Whole train set size:', train_dataset.__len__())

train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=batch_size,
                                            num_workers=workers,
                                            shuffle=True,
                                            pin_memory=True)

data_transforms_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])

val_dataset = FERPlusDataset(val_csv, phase='val', transform=data_transforms_val)
print('Validation set size:', val_dataset.__len__())

val_loader = torch.utils.data.DataLoader(val_dataset,
                                            batch_size=batch_size,
                                            num_workers=workers,
                                            shuffle=False,
                                            pin_memory=True)

criterion_cls = torch.nn.CrossEntropyLoss()

params = list(model.parameters())
optimizer = torch.optim.SGD(params, lr=lr, weight_decay=1e-4, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

best_acc = 0
for epoch in tqdm(range(1, epochs + 1)):
    running_loss = 0.0
    correct_sum = 0
    iter_cnt = 0
    model.train()

    for imgs, targets in train_loader:
        iter_cnt += 1
        optimizer.zero_grad()

        imgs = imgs.to(device)
        targets = targets.to(device)
        out, _, _ = model(imgs, torch.zeros(imgs.size(0), 39936).to(device)) # Pass None for latent

        loss = criterion_cls(out, targets)

        loss.backward()
        optimizer.step()

        running_loss += loss
        _, predicts = torch.max(out, 1)
        correct_num = torch.eq(predicts, targets).sum()
        correct_sum += correct_num

    acc = correct_sum.float() / float(train_dataset.__len__())
    running_loss = running_loss / iter_cnt
    tqdm.write('[Epoch %d] Training accuracy: %.4f. Loss: %.3f. LR %.6f' % (epoch, acc, running_loss, optimizer.param_groups[0]['lr']))

    with torch.no_grad():
        running_loss = 0.0
        iter_cnt = 0
        bingo_cnt = 0
        sample_cnt = 0
        baccs = []

        y_true = []
        y_pred = []

        model.eval()
        for imgs, targets in val_loader:
            imgs = imgs.to(device)
            targets = targets.to(device)

            out, _, _ = model(imgs, torch.zeros(imgs.size(0), 39936).to(device))   # Pass None for latent
            loss = criterion_cls(out, targets)
            running_loss += loss
            iter_cnt += 1
            _, predicts = torch.max(out, 1)
            correct_num = torch.eq(predicts, targets)
            bingo_cnt += correct_num.sum().cpu()
            sample_cnt += out.size(0)
            y_true.append(targets.cpu().numpy())
            y_pred.append(predicts.cpu().numpy())

            baccs.append(balanced_accuracy_score(targets.cpu().numpy(), predicts.cpu().numpy()))
        running_loss = running_loss / iter_cnt
        scheduler.step()

        acc = bingo_cnt.float() / float(sample_cnt)
        acc = np.around(acc.numpy(), 4)
        best_acc = max(acc, best_acc)

        y_true = np.concatenate(y_true)
        y_pred = np.concatenate(y_pred)

        bacc = np.around(np.mean(baccs), 4)
        tqdm.write("[Epoch %d] Validation accuracy:%.4f. bacc:%.4f. Loss:%.3f" % (epoch, acc, bacc, running_loss))
        tqdm.write("best_acc:" + str(best_acc))

/root/miniconda3/envs/fer/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/fer/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_3634410/2602754637.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to

AffectNet weights loaded successfully.
Unique labels in dataset after filtering: [0 1 2 3 4 5 6 7]
Whole train set size: 25045
Unique labels in dataset after filtering: [0 1 2 3 4 5 6 7]
Validation set size: 3191


  0%|          | 0/60 [00:50<?, ?it/s]

[Epoch 1] Training accuracy: 0.3567. Loss: 1.585. LR 0.100000


  2%|▏         | 1/60 [00:53<52:26, 53.34s/it]

...

[Epoch 59] Validation accuracy:0.8389. bacc:0.6863. Loss:0.451
best_acc:0.8427


 98%|█████████▊| 59/60 [47:21<00:47, 47.30s/it]

[Epoch 60] Training accuracy: 0.8686. Loss: 0.373. LR 0.000001


/root/miniconda3/envs/fer/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/root/miniconda3/envs/fer/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
100%|██████████| 60/60 [47:24<00:00, 47.40s/it]

[Epoch 60] Validation accuracy:0.8355. bacc:0.6845. Loss:0.452
best_acc:0.8427


In [4]:
def evaluate_model(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    y_true = []
    y_pred = []

    print("Starting evaluation...")
    with torch.no_grad():  # Disable gradient computation
        for i, (imgs, targets) in enumerate(test_loader):

            imgs = imgs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs, _, _ = model(imgs, torch.zeros(imgs.size(0), 39936).to(device))  # Provide dummy latent input
            _, predictions = torch.max(outputs, 1)  # Get predicted class

            # Collect results
            correct += (predictions == targets).sum().item()
            total += targets.size(0)

            y_true.extend(targets.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())

    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0  # Prevent division by zero
    print(f"Test Accuracy: {accuracy}")  
    print(f"Test Accuracy: {accuracy}")
    print(f"Test Accuracy: {accuracy}")

    return accuracy
data_transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])

test_dataset = FERPlusDataset(test_csv, phase='test', transform=data_transforms_test)
print('Test set size:', test_dataset.__len__())

test_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size=batch_size,
                                            num_workers=workers,
                                            shuffle=False,
                                            pin_memory=True)

# Evaluate model
acc = evaluate_model(model, test_loader, device)
print(f"Final Test Accuracy: {acc:.4f}")


Unique labels in dataset after filtering: [0 1 2 3 4 5 6 7]
Test set size: 3136
Starting evaluation...
Test Accuracy: 0.8447066326530612
Test Accuracy: 0.8447066326530612
Test Accuracy: 0.8447066326530612
Final Test Accuracy: 0.8447


# MobilNet Latent dlib attention

In [1]:
from torch import nn
from torch.nn import functional as F
import torch
import torch.nn.init as init
from torchvision.models import mobilenet_v2
from torchvision import transforms
from torchvision import models

from sklearn.metrics import balanced_accuracy_score
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.metrics")



In [2]:
from torch import nn
from torch.nn import functional as F
import torch
import torch.nn.init as init
from torchvision import models

class ModifiedOURS(nn.Module):
    def __init__(self, num_class=7, num_head=4, pretrained=True):
        super(ModifiedOURS, self).__init__()

        # MobileNetV2 백본
        mobilenet = models.mobilenet_v2(pretrained=pretrained)
        self.features = mobilenet.features  # MobileNetV2의 특징 추출 레이어

        self.num_head = num_head
        for i in range(num_head):
            setattr(self, f"cat_head{i}", CrossAttentionHead(1280))  # 채널 크기를 1280으로 수정

        # Self-Attention
        self.self_attention = SelfAttention(1280)

        # 분류기
        self.fc = nn.Linear(1280, 256)
        self.fc2 = nn.Linear(256, 128)
        self.batch_norm = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, num_class)
        self.bn = nn.BatchNorm1d(num_class)

    def forward(self, x, latent):
        # MobileNetV2 특징 추출
        x = self.features(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))  # GAP(Global Average Pooling)
        x = x.view(x.size(0), -1)  # (batch_size, 1280)

        # Cross Attention Heads
        heads = []
        for i in range(self.num_head):
            heads.append(getattr(self, f"cat_head{i}")(x.unsqueeze(-1).unsqueeze(-1)))  # 차원 추가

        heads = torch.stack(heads).permute([1, 0, 2])  # (batch_size, num_heads, features)
        heads = heads.sum(dim=1)  # (batch_size, features)

        # Self-Attention
        attention_out = self.self_attention(heads)

        # 분류기
        out = self.fc(attention_out)
        out = self.fc2(out)
        out = self.batch_norm(out)
        out = self.fc4(out)
        out = self.bn(out)

        return out, x, heads


class CrossAttentionHead(nn.Module):
    def __init__(self, channel_dim):
        super().__init__()
        self.sa = SpatialAttention(channel_dim)
        self.ca = ChannelAttention(channel_dim)
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x):
        ca = self.ca(x)
        sa = self.sa(ca)
        return sa


class SpatialAttention(nn.Module):
    def __init__(self, channel_dim):
        super().__init__()
        self.conv1x1 = nn.Sequential(
            nn.Conv2d(channel_dim, channel_dim // 2, kernel_size=1),
            nn.BatchNorm2d(channel_dim // 2),
        )
        self.conv_3x3 = nn.Sequential(
            nn.Conv2d(channel_dim // 2, channel_dim, kernel_size=3, padding=1),
            nn.BatchNorm2d(channel_dim),
        )
        self.relu = nn.ReLU()
        self.gap = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        y = self.conv1x1(x)
        y = self.conv_3x3(y)
        y = self.relu(y)
        y = y.sum(dim=1, keepdim=True)

        out = x * y
        out = self.gap(out)
        out = out.view(out.size(0), -1)
        return out


class ChannelAttention(nn.Module):
    def __init__(self, channel_dim):
        super().__init__()
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.attention = nn.Sequential(
            nn.Linear(channel_dim, channel_dim // 4),
            nn.BatchNorm1d(channel_dim // 4),
            nn.ReLU(inplace=True),
            nn.Linear(channel_dim // 4, channel_dim),
            nn.Sigmoid()
        )

    def forward(self, sa):
        sa2 = self.gap(sa)
        sa2 = sa2.view(sa2.size(0), -1)
        y = self.attention(sa2)
        y = y.unsqueeze(dim=-1).unsqueeze(dim=-1)

        out = sa * y
        return out


class SelfAttention(nn.Module):
    def __init__(self, embed_dim):
        super(SelfAttention, self).__init__()
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)

        attention_scores = torch.matmul(Q, K.transpose(-2, -1))
        attention_scores = self.softmax(attention_scores)

        attention_output = torch.matmul(attention_scores, V)
        return attention_output


In [3]:
import dlib
import pandas as pd
import numpy as np
from PIL import Image
import cv2
from torch.utils.data import Dataset
from tqdm import tqdm
class FERPlusDataset(Dataset):
    def __init__(self, data_csv, phase, transform=None):
        self.phase = phase
        self.transform = transform

        # Read the dataset CSV file
        self.data = pd.read_csv(data_csv)
        self.data.iloc[:, 2:12] = self.data.iloc[:, 2:12].replace(1, 0)

        # Get file paths and labels
        self.file_paths = self.data.iloc[:, 0].values
        self.counts = self.data.iloc[:, 2:12].values  # 감정 점수들

        # Apply constraints to filter valid samples
        self._apply_constraints()

        # Use argmax to determine the emotion class
        self.labels = np.argmax(self.counts, axis=1)

        # Debugging: Check label range
        print("Unique labels in dataset after filtering:", np.unique(self.labels))

        # Dlib face detector and predictor initialization
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor('/root/FER2013/shape_predictor_68_face_landmarks.dat')  # Dlib 모델 파일 경로 필요

    def _apply_constraints(self):
        # Constraint : 'unknown-face' 또는 'not-face' 레이블 제거
        max_counts = self.counts.max(axis=1)
        counts_eq_max = (self.counts == max_counts[:, None])
        constraint1_violation = counts_eq_max[:, [8, 9]].any(axis=1)

        # Constraint : 최대 투표 수를 가진 레이블이 3개 초과 제거
        num_max_labels = counts_eq_max.sum(axis=1)
        constraint2_violation = num_max_labels > 3

        # Constraint : 최대 투표 수가 전체 투표 수의 절반 이하인 경우 제거
        total_votes = self.counts.sum(axis=1)
        constraint3_violation = max_counts <= (total_votes / 2)

        # Combine constraints
        valid_samples = ~(
            constraint1_violation | constraint2_violation | constraint3_violation
        )

        # Apply valid samples filter
        self.file_paths = self.file_paths[valid_samples]
        self.counts = self.counts[valid_samples]

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        if self.phase == 'train':
            path = '/data/FER2013/FER2013Train/' + self.file_paths[idx]
        elif self.phase == 'val':
            path = '/data/FER2013/FER2013Valid/' + self.file_paths[idx]
        elif self.phase == 'test':
            path = '/data/FER2013/FER2013Test/' + self.file_paths[idx]
        
        # Open image
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)  # Convert grayscale to RGB by duplicating the channel
        
        # Align face using Dlib
        image = self._align_face(image)

        # Convert to PIL image for further processing
        image = Image.fromarray(image)

        label = self.labels[idx]

        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def _align_face(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        faces = self.detector(gray)

        if len(faces) == 0:
            return image  # 얼굴이 감지되지 않으면 원본 이미지를 반환

        for face in faces:
            landmarks = self.predictor(gray, face)

            # 좌우 눈의 중심 좌표 추출
            left_eye = (landmarks.part(36).x, landmarks.part(36).y)
            right_eye = (landmarks.part(45).x, landmarks.part(45).y)

            # 두 눈의 중심 계산
            eye_center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)

            # 눈 사이의 기울기 계산
            delta_x = right_eye[0] - left_eye[0]
            delta_y = right_eye[1] - left_eye[1]
            angle = np.degrees(np.arctan2(delta_y, delta_x))

            # 회전 행렬 계산
            rot_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)

            # 이미지 회전 및 정렬
            aligned_face = cv2.warpAffine(image, rot_matrix, (image.shape[1], image.shape[0]))

            return aligned_face  # 첫 번째 얼굴만 처리

        return image

In [4]:


# Define variables
train_csv = '/data/FER2013/train_label.csv'
val_csv = '/data/FER2013/valid_label.csv'
test_csv = '/data/FER2013/test_label.csv'
batch_size = 128
lr = 0.001
workers = 4
epochs = 60

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = True

latent_dim = 1280  # MobileNetV2의 출력 채널 크기
latent_input = latent_dim  # `latent` 입력 크기 수정

model = ModifiedOURS(num_class=8 ,num_head=4)# YOUR MODEL CLASS SHOULD BE DEFINED ELSEWHERE
model.to(device)

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([
            transforms.RandomRotation(20),
            transforms.RandomCrop(224, padding=32)
        ], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(scale=(0.02, 0.25)),
])

train_dataset = FERPlusDataset(train_csv, phase='train', transform=data_transforms)
print('Whole train set size:', train_dataset.__len__())

train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=batch_size,
                                            num_workers=workers,
                                            shuffle=True,
                                            pin_memory=True)

data_transforms_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])

val_dataset = FERPlusDataset(val_csv, phase='val', transform=data_transforms_val)
print('Validation set size:', val_dataset.__len__())

val_loader = torch.utils.data.DataLoader(val_dataset,
                                            batch_size=batch_size,
                                            num_workers=workers,
                                            shuffle=False,
                                            pin_memory=True)

# Loss 함수 정의
def stable_cross_entropy_loss(output, targets, epsilon=1e-7):
    output = F.log_softmax(output, dim=1).clamp(min=epsilon, max=1 - epsilon)
    return F.nll_loss(output, targets)

params = list(model.parameters())
# 손실 함수 수정
criterion_cls = stable_cross_entropy_loss

# 학습률 및 옵티마이저 수정
optimizer = torch.optim.SGD(params, lr=0.01, weight_decay=1e-4, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
best_acc = 0
latent_dim = 1280  # MobileNetV2 출력 채널 크기
for epoch in tqdm(range(1, epochs + 1)):
    running_loss = 0.0
    correct_sum = 0
    iter_cnt = 0
    model.train()

    for imgs, targets in train_loader:
        iter_cnt += 1
        optimizer.zero_grad()

        imgs, targets = imgs.to(device), targets.to(device)
        latent_input = torch.zeros(imgs.size(0), latent_dim).to(device)  # `latent` 초기화
        out, _, _ = model(imgs, latent_input)

        # 안정적인 CrossEntropyLoss 적용
        loss = stable_cross_entropy_loss(out, targets)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicts = torch.max(out, 1)
        correct_sum += (predicts == targets).sum().item()

    acc = correct_sum / len(train_dataset)
    running_loss /= iter_cnt
    tqdm.write(f'[Epoch {epoch}] Training accuracy: {acc:.4f}. Loss: {running_loss:.3f}. LR {optimizer.param_groups[0]["lr"]:.6f}')

    # Validation
    with torch.no_grad():
        running_loss = 0.0
        iter_cnt = 0
        correct_sum = 0
        total_samples = 0
        baccs = []

        y_true = []
        y_pred = []

        model.eval()
        for imgs, targets in val_loader:
            imgs, targets = imgs.to(device), targets.to(device)
            latent_input = torch.zeros(imgs.size(0), latent_dim).to(device)  # `latent` 초기화
            out, _, _ = model(imgs, latent_input)

            loss = stable_cross_entropy_loss(out, targets)
            running_loss += loss.item()
            iter_cnt += 1

            _, predicts = torch.max(out, 1)
            correct_sum += (predicts == targets).sum().item()
            total_samples += targets.size(0)

            y_true.append(targets.cpu().numpy())
            y_pred.append(predicts.cpu().numpy())

            baccs.append(balanced_accuracy_score(targets.cpu().numpy(), predicts.cpu().numpy()))

        val_acc = correct_sum / total_samples
        running_loss /= iter_cnt
        scheduler.step()

        y_true = np.concatenate(y_true)
        y_pred = np.concatenate(y_pred)
        bacc = np.mean(baccs)

        tqdm.write(f'[Epoch {epoch}] Validation accuracy: {val_acc:.4f}. bacc: {bacc:.4f}. Loss: {running_loss:.3f}')
        tqdm.write(f'Best validation accuracy: {best_acc:.4f}')


/root/miniconda3/envs/fer/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/fer/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Unique labels in dataset after filtering: [0 1 2 3 4 5 6 7]
Whole train set size: 25045
Unique labels in dataset after filtering: [0 1 2 3 4 5 6 7]
Validation set size: 3191


TypeError: stable_cross_entropy_loss() missing 2 required positional arguments: 'output' and 'targets'

In [ ]:
def evaluate_model(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    y_true = []
    y_pred = []

    print("Starting evaluation...")
    with torch.no_grad():  # Disable gradient computation
        for i, (imgs, targets) in enumerate(test_loader):

            imgs = imgs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs, _, _ = model(imgs, torch.zeros(imgs.size(0), 39936).to(device))  # Provide dummy latent input
            _, predictions = torch.max(outputs, 1)  # Get predicted class

            # Collect results
            correct += (predictions == targets).sum().item()
            total += targets.size(0)

            y_true.extend(targets.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())

    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0  # Prevent division by zero
    print(f"Test Accuracy: {accuracy}")  
    print(f"Test Accuracy: {accuracy}")
    print(f"Test Accuracy: {accuracy}")

    return accuracy
data_transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])

test_dataset = FERPlusDataset(test_csv, phase='test', transform=data_transforms_test)
print('Test set size:', test_dataset.__len__())

test_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size=batch_size,
                                            num_workers=workers,
                                            shuffle=False,
                                            pin_memory=True)

# Evaluate model
acc = evaluate_model(model, test_loader, device)
print(f"Final Test Accuracy: {acc:.4f}")
